## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/134bert-base-last/' + '*.h5')
model_path_list += glob.glob('/kaggle/input/134bert-base-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_1.h5
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_2.h5
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_3.h5
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_4.h5
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_5.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_1_last_epoch.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_2_last_epoch.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_3_last_epoch.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_4_last_epoch.h5
/kaggle/input/134bert-base-last/134-BERT_base_unc raw_model_fold_5_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.940972,0.596343,0.241477,0.480413,0.554753,0.542764,0.671243,0.639646,0.596047,...,0.881290,0.902363,0.560415,0.958325,0.956009,0.813036,0.061325,0.044388,0.863647,0.899138
1,46,0.875116,0.531219,0.006457,0.780044,0.749331,0.914381,0.570885,0.472141,0.050045,...,0.715897,0.952450,0.664338,0.972603,0.981494,0.880661,0.913572,0.089868,0.077800,0.879255
2,70,0.918377,0.653706,0.034029,0.808162,0.840070,0.929594,0.592973,0.474078,0.124426,...,0.852197,0.947315,0.651805,0.973378,0.978291,0.875716,0.094875,0.054910,0.883543,0.925229
3,132,0.885727,0.398305,0.007030,0.693097,0.772405,0.910556,0.553407,0.452791,0.114537,...,0.690875,0.954458,0.669750,0.971244,0.982876,0.895561,0.839454,0.136631,0.522788,0.912796
4,200,0.933089,0.426324,0.050186,0.826160,0.704594,0.809998,0.650251,0.583839,0.098154,...,0.678186,0.921645,0.627128,0.969953,0.969521,0.844307,0.122369,0.104623,0.666136,0.906068


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.890709,0.591226,0.034239,0.730511,0.804835,0.853341,0.577364,0.489078,0.225599,...,0.789907,0.935983,0.664254,0.963918,0.973960,0.868169,0.502242,0.130474,0.526307,0.909205
std,2812.670060,0.046117,0.130710,0.058733,0.109360,0.107359,0.128990,0.050269,0.084749,0.194002,...,0.083818,0.023532,0.044739,0.011893,0.011094,0.039043,0.320352,0.063407,0.280191,0.021231
min,39.000000,0.758047,0.348963,0.002865,0.275027,0.316415,0.255742,0.472329,0.327897,0.008489,...,0.591385,0.806346,0.541766,0.909131,0.910164,0.730030,0.008920,0.008956,0.027752,0.836440
25%,2572.000000,0.859816,0.471296,0.007544,0.667873,0.764492,0.826935,0.541604,0.427763,0.079955,...,0.720101,0.923336,0.634148,0.955959,0.968627,0.844939,0.160591,0.085005,0.283124,0.896382
50%,5093.000000,0.893522,0.590873,0.011929,0.733831,0.818136,0.904939,0.568301,0.466475,0.150214,...,0.794036,0.940562,0.662897,0.965629,0.976824,0.874278,0.570493,0.130530,0.544575,0.911930
75%,7482.000000,0.927029,0.698765,0.029548,0.811629,0.878727,0.930597,0.611259,0.534097,0.326408,...,0.864545,0.953137,0.694175,0.972799,0.981853,0.895417,0.796299,0.172839,0.772648,0.925255
max,9640.000000,0.978865,0.906623,0.511647,0.962141,0.977884,0.973573,0.710960,0.750351,0.788330,...,0.958556,0.978882,0.804546,0.986292,0.992244,0.958387,0.952133,0.395738,0.986737,0.950761
